# Deploy

### The GET Method

**GET is used to request data from a specified resource.**

GET is one of the most common HTTP methods.

Note that the query string (name/value pairs) is sent in the URL of a GET request:

>/test/demo_form.php?name1=value1&name2=value2


* GET requests can be cached
* GET requests remain in the browser history
* GET requests can be bookmarked
* GET requests should never be used when dealing with sensitive data
* GET requests have length restrictions
* GET requests are only used to request data (not modify)

# Flask:

Nos permite apificar nuestro modelo, en realidad cualquier script de python. Podemos enviar un GET requests a algun puerto de nuestra computadora. Ese requests puede contener los parametros que luego el script va usar.

En este ejemplo, una simple app que imprime el cuadrado de la variable 'a' si es que existe. Para que funcione deben guardar el siguiente codigo en un archivo 'prueba.py' y correrlo en la consola `python prueba.py`:

```python
from flask import Flask, jsonify, request

app = Flask('Servidor Get')
@app.route('/',methods=['GET'])
def hola():
   
    # obtengo los datos del request get.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data=request.args.to_dict()
    try:            
        resp='el cuadrado de a es : '+str(int(data['a'])*int(data['a']))
    except:
        resp='no se envio la variable a'
    return(resp)

app.run(host='0.0.0.0',  port=5002 )
```

```python
from flask import Flask, jsonify, request

app = Flask('Servidor Get')
@app.route('/',methods=['GET'])

def hola():
   
    # obtengo los datos del request get.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data=request.args.to_dict()
    try:            
        resp='el cuadrado de a es : '+str(int(data['a'])*int(data['a']))
    except:
        resp='no se envio la variable a'
    return(resp)

app.run(host='0.0.0.0',  port=5002 )
```

* http://127.0.0.1:5002/?a=4

* http://127.0.0.1:5002/?a=4&b='asdfasf'

* http://deploy.myftp.org:5002/?c=4&b='asdfasf'

* http://127.0.0.1:5002/?c=4&a=['1','12']



### The POST Method

**POST is used to send data to a server to create/update a resource.**

>The data sent to the server with POST is stored in the request body of the HTTP request:

```
POST /test/demo_form.php HTTP/1.1
Host: w3schools.com
name1=value1&name2=value2
POST is one of the most common HTTP methods.
```

Some other notes on POST requests:

* POST requests are never cached
* POST requests do not remain in the browser history
* POST requests cannot be bookmarked
* POST requests have no restrictions on data length

Queremos pasar la variable 'a' como un vector, con el metodo POST es muy sencillo.

In [12]:
import requests as req
import json

# parametros, la url y un diccionario
url='http://127.0.0.1:5001/' # <--- url por default en servidor
url2='http://deploy.myftp.org:5002/' # <--- url publica de mi ec2

data = { 'a': ['1', '-2', '4', '13', '5', '0', '10', '1'] ,
          'b' : 2342 }

# el requests
r = req.post(url2, json.dumps(data))
r

ConnectionError: HTTPConnectionPool(host='deploy.myftp.org', port=5002): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x103872e50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [22]:
r.json()

{'response': [1, 4, 16, 169, 25, 0, 100, 1]}

In [23]:
r.json()['response']

[1, 4, 16, 169, 25, 0, 100, 1]

In [24]:
type(r.json()['response'])

list

In [14]:
from flask import Flask, jsonify, request
app = Flask('Predictor de examenes')
#aclaramos la ruta del recurso y el tipo: (podriamos poner /modelos/listener)
@app.route('/',methods=['POST'])
#solo con esta linea, ya tenemos un recurso recibiendo requests en "127.0.0.1/".

#defino la funcion que manejarara el request
def predict():
    # obtengo los datos del request post.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data = request.get_json(force=True)
    try:
        a_vector = np.array(data['a']).astype('int')
        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        a_2=a_vector**2
        #importante pasar a lista     
        resp={'response' : a_2.tolist() }
    except:
        print('no mandaron a')
        resp={'response' : 'no esta presente la variable a'}
       
        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # y respondemos el request con un json mediante este return
        # este json es incorporado en el cuerpo de la respuesta
    return jsonify(resp)

# Ok... algo un poco mas interactivo, no hay?

```python
from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.resources import INLINE
from flask import Flask, jsonify, request, render_template
import numpy as np
from bokeh.models import Range1d

app = Flask('Ejemplo Bokeh')
@app.route('/',methods=['GET'])
def bokeh():
    data=request.args.to_dict()
    a=int(data['a'])
    b=int(data['b'])
    n=int(data['n'])
    x=np.linspace(0,10,n)
    y=a*np.sin(b*x)
    fig = figure(plot_width=1280, plot_height=600)
    fig.line(x,y)
    left, right, bottom, top = 0, 10, -10, 10
    fig.x_range=Range1d(left, right)
    fig.y_range=Range1d(bottom, top)
    # magia html del bokeh
    js_resources = INLINE.render_js()
    css_resources = INLINE.render_css()
    script, div = components(fig)
    html = render_template(
        'index.html',
        plot_script=script,
        plot_div=div,
        js_resources=js_resources,
        css_resources=css_resources,
    )
    return html.encode('utf8')
app.run(host='0.0.0.0',  port=5002)
```

Esta app de Flask genera un servidor de Bokeh, con el cual podemos interactuar!
Es un ploteo de una funcion trigonometrica definida segun:

$$ x =  (x_0,x_1,x_2,...,x_n) $$
$$ f(x) = a \cdot sen(b \cdot x ) $$


*   http://127.0.0.1:5002/?a=2&b=10&n=1000
*   http://127.0.0.1:5002/?a=2&b=50&n=1000
*   http://127.0.0.1:5002/?a=2&b=80&n=10000
*   http://deploy.myftp.org:5002/?a=2&b=80&n=10000

## Y dash? Eso que plotly me quiere vender todo el tiempo?


https://plotly.com/dash/


# Streamlit

* Flask tiene la gran desventaja que solo se puede interactuar mediante requests.

* Streamlit es un framework para crear web-apps de una manera super sencilla y dinámica.

* Todos los elementos de una página web son definidos en un **script de python**. Cada uno será definido por una función y a medida que los agregamos podemos ver su resultado en una página de nuestro navegador.


![](https://raw.githubusercontent.com/streamlit/docs/main/public/images/Streamlit_overview.gif)




<video width="720"  controls>
  <source src="https://s3-us-west-2.amazonaws.com/assets.streamlit.io/videos/hero-video.mp4" type="video/mp4">
</video>

```python
import streamlit as st

x=st.slider('eleji valor')

st.write('el cubo de ',x , ' es', x*x*x)
```

![](1.png)

https://www.streamlit.io/


* Las apps de Streamlit son scripts de python que se ejecutan secuencialmente. 

* Cada vez que un usuario accede a la app, el script se re ejecuta.

* Duran la ejeccion, Streamlit muestra el output en el navegador.

* Cada interaccion con un widget hace que el script se re-ejecute con el nuevo valor del widget.

* Se puede usar **cache** para evitar recalcular funciones muy pesadas, mejorando el tiempo de respuesta de la app.


## Widgets

Los widgets son objetos que sirve para que el usuario interactue con los datos o el modelo, hay una extensa lista:

https://docs.streamlit.io/en/stable/api.html

Ejemplo:

```python
import streamlit as st
x = st.slider('x')  # 👈 widget
st.write(x, 'squared is', x * x)
```

En este ejmeplo la app tira un output “0 squared is 0”. Cada vez que el usuario interactua , Streamlit vuelve a ejecutar el script entero, asignando el nuevo valor a la variable asociadad al widget. Si fuera 10, Streamlit corre el codigo y daria como output “10 squared is 100”.

## Componentes 

Extienden la funcionalidad de Streamlit.

https://www.streamlit.io/components

```python
import streamlit as st
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure

# widget de latex
st.latex('f(x) = a\sin( \omega  x)')

# widget input numerica
a = st.number_input('Insert a',1)

# texto
st.write('a = ', a)

#widget de sliders
b = st.slider('w', 0, 10, 1)
n = st.slider('n', 0, 500, 25) 

# python 
x=np.linspace(0,10,int(n))
y=a*np.sin(b*x)
p = figure(
     title='plot',
     x_axis_label='x',
     y_axis_label='y')
p.line(x, y, legend_label='sin(x)', line_width=2)
p.circle(x, y, legend_label='sin(x)', line_width=2)

#widget de bokeh
st.bokeh_chart(p, use_container_width=True)
```

Guardamos todo el codigo en un archivo ".py" y luego lo ejecutamos desde la consola:
    
```shell
streamlit run miarchivito.py
```


http://3.138.190.237:8501/

http://deploy.myftp.org:8501

### muy bonito! pero como se lo muestro al CLIENTE/JEFE/MIVIEJA? 


* Firewalls

* Llamar a fibertel para que abran puertos o habiliten una DMZ

* Peligroso

* Siempre tiene que estar corriendo la terminal de python



# Cloud computing

![](2.png)

![](3.png)

### Heroku

* Gratis, sin tarjeta de credito a diferencia de AWS

* Se integra muy bien con GITHUB

Una vez que tenemos nuestra app de streamlit funcionando en nuestra compu, creamos un repo en github y subimos el ".py" con nuestra app y 3 archivos mas:

```shell
app_st.py
requirements.txt
runtime.txt
Procfile
create_config.sh
```


![](4.png)


![](5.png)

[Repo](https://github.com/carabedo/properatti)
[App](https://geopami.herokuapp.com/)

# IAAS (infrastructure-as-a-service)


![](iaas.jpg)




## Virtual Machines


Ejemplo en oracle:

Creamos una instancia [aca](https://cloud.oracle.com/compute/instances?region=sa-vinhedo-1)


## ssh


Le cambiamos los permisos a la llave que nos bajamos:

`chmod 400 <private_key_file>`

En mi caso:

`ssh -i sshtest.key` 

Nos conectamos a una terminal de la maquina virtual usando la llave:

`ssh -i <private_key_file> <username>@<public-ip-address>`

En mi caso:

` ssh -i sshtest.key opc@193.123.101.232`      

[tutorial de oracle](https://docs.oracle.com/en-us/iaas/Content/Compute/Tasks/accessinginstance.htm)

## ssh

Creemos una carpeta:

`mkdir prueba`

Entremos en la carpeta y creemos un archivo con codigo en python

```
    cd prueba
    vim prueba.py
```

Ejecutemos nuestro script:

`python3 prueba.py`

## ssh con visualcode

![](https://code.visualstudio.com/assets/docs/remote/ssh-tutorial/remote-ssh-extension.png)



![](https://code.visualstudio.com/assets/docs/remote/ssh-tutorial/remote-commands.png)

Hay que editar el archivo que guarda la direccion de las maquinas virtuales y la ubicacion en nuestra compu de las llaves. En Host ponemos el nombre que queremos, HostName es la ip de la maquina, luego el path de la .key en nuestra compu y por ultimo el nombre del usuario en la maquina virtual para las vms de oracle es `opc`.

```
Host Oracle
  HostName 193.123.110.190
  IdentityFile /Users/fernando/gits/deploy/sshtest.key
  User opc
```

No importa en que plataforma tenemos nuestra maquina virtual, solo necesitamos la ip publica, una llave y saber el nombre de nuestro usuario.

# Serverless
![](6.png)

# lambdas (aws)

### api gateway
### layers

https://github.com/keithrozario/Klayers

### test&deploy

In [81]:

data={
  "key1": "culo",
  "key2": "value2",
  "key3": "value3"
}

#sin esto la lambda recibe el dict en base64 

headers = {'Content-type': 'application/json'}

test=req.post('https://6xpc1u0n12.execute-api.us-east-2.amazonaws.com/default/test2',data=data,headers=headers)
test

<Response [200]>

In [82]:
test.json()

[{'body': 'key1=culo&key2=value2&key3=value3',
  'y': [4, 9, 16, 25, 36, 49],
  'name': 'cuadratica'}]

In [83]:
test.json()[0]['body']

'key1=culo&key2=value2&key3=value3'

In [88]:
s=test.json()[0]['body']
s

'key1=culo&key2=value2&key3=value3'

In [90]:
s.split('&')

['key1=culo', 'key2=value2', 'key3=value3']

In [27]:
result=test.json()

result[0]['y']

[4, 9, 16, 25, 36, 49]

In [24]:
type(result[0]['y']),type(result[0]['y'][0])

(list, int)